In [ ]:
import requests
def get_api_key(email: str) -> str:
    response = requests.post(
        "http://127.0.0.1:8000/generate-key/",
        json={"email": email}
    )
    if response.status_code == 200:
        return response.json()["api_key"]
    else:
        raise Exception(f"Error: {response.status_code}\n{response.text}")

key = get_api_key("camila@ejemplo.com")
print("Tu API Key es:", key)


Tu API Key es: 2g5W63ZCG-kdVMRgbEJ0WFyBWjrQLmOcU49iNjOZ


codigo de mapas.ipynb

In [ ]:
def load_geospatial_data(poi_path, street_path, voronoi_path):
    """
    Load the three GeoJSON layers with appropriate error handling
    """
    print("Loading GeoJSON data files...")

    try:
        # Load only 2000 POIs
        pois = gpd.read_file(poi_path, rows=2000, low_memory=False)  # <- Añade rows=2000
        print(f"Loaded POIs: {len(pois)} records")
        
        # Check if we need to convert POIs from LineString to Point
        if 'LineString' in str(pois.geometry.iloc[0]):
            print("Converting POI LineStrings to Points (using first point of each line)...")
            pois['orig_geometry'] = pois.geometry  # Keep original geometry
            # Extract the first point from each LineString
            pois['geometry'] = pois.geometry.apply(lambda line: Point(line.coords[0]))
    
        # Load street Voronoi polygons
        streets = gpd.read_file(street_path, low_memory=False)
        print(f"Loaded streets: {len(streets)} records")
        
        # Load building polygons - use a separate file if available
        voronoi = gpd.read_file(voronoi_path, low_memory=False)
        print(f"Loaded Voronoi polygons: {len(voronoi)} records")
        
        # Ensure all are in the same coordinate system
        if pois.crs != streets.crs:
            print("Converting street data to match POI CRS...")
            streets = streets.to_crs(pois.crs)
        
        if pois.crs != voronoi.crs:
            print("Converting Voronoi data to match POI CRS...")
            voronoi = voronoi.to_crs(pois.crs)
            
        # Assign common CRS if any is missing
        if pois.crs is None:
            print("Setting CRS for POIs to EPSG:4326")
            pois.set_crs(epsg=4326, inplace=True)
            
        if streets.crs is None:
            print("Setting CRS for streets to EPSG:4326")
            streets.set_crs(epsg=4326, inplace=True)
            
        if voronoi.crs is None:
            print("Setting CRS for Voronoi to EPSG:4326")
            voronoi.set_crs(epsg=4326, inplace=True)

        # Convert to projected CRS for Mexico (UTM zone 14N)
        target_crs = "EPSG:32614"
        print(f"Converting all layers to projected CRS: {target_crs}")
        pois = pois.to_crs(target_crs)
        streets = streets.to_crs(target_crs)
        voronoi = voronoi.to_crs(target_crs)
            
        return pois, streets, voronoi
        
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        raise